In [ ]:
from typing import List
from pathlib import Path

import report as reports
import augment
import plots

# for plotting
import matplotlib.pyplot as plt
import seaborn as sbn
import pandas as pnd

# Load latest report

In [ ]:
current = reports.load_report(Path('../../reports/current-09-09'))
report = reports.make_report_data_frame(current)
report = augment.augment_report_data_frame(report)

report

# Print basic stats

In [ ]:
print(f'reports: {len(current)}')

functions = sorted(set((rep.function for rep in current)))
print(f'functions: {len(functions)}')

targets = sorted(set((rep.target for rep in current)))
print(f'targets: {targets}')

dimensions = sorted(set((rep.dimension for rep in current)))
print(f'dimensions: {dimensions}')

batch_sizes = sorted(set((rep.batch_size for rep in current)))
print(f'batch_sizes: {batch_sizes}')

# Plot improvements over all functions

In [ ]:
_ = plots.delta(report)

# Aggregation

In [ ]:
_ = plots.aggregation(report)

# Experiments

In [ ]:
data = report.groupby(by=['dimension', 'batch_size', 'target']).aggregate({'std_dev': 'mean'}).reset_index()

fig, ax = plt.subplots(1, 2, figsize=(12, 7))

ax[0].set_title('dimension 2')
sbn.barplot(data[data['dimension'] == 2], x='batch_size', y='std_dev', hue='target', ax=ax[0])

ax[1].set_title('dimension 40')
sbn.barplot(data[data['dimension'] == 40], x='batch_size', y='std_dev', hue='target', ax=ax[1])

In [ ]:
# target | function | avg_value

triplets = [(b.target, b.function, b.dimension, b.batch_size, b.measurements.avg_values) for b in current]
count = sum((len(t[4]) for t in triplets))
entries = list()
for target, function, dimension, batch_size, avg_values in triplets:
    for value in avg_values:
        entries.append((target, function, dimension, batch_size, value))
    
data = pnd.DataFrame(entries, columns=['target', 'function', 'dimension', 'batch_size', 'avg_value'])
data = data[(data['dimension'] == 40) & (data['batch_size'] == 64)]
data = data[(data['function'] == 'Sphere') | (data['function'] == 'Ellipsoid') | (data['function'] == 'Rastrigin')]

sbn.set(rc={'figure.figsize': (8, 6)})
sbn.boxplot(data=data, x='function', y='avg_value', hue='target')